In [46]:
import pandas as pd
import numpy as np
import re

In [47]:
df = pd.read_excel(r"C:\Users\User\Documents\horse-racing-arima-kinen-2013\data\raw\results.xlsx")

In [48]:
horse_names = df['Horse'].unique().tolist()
print(horse_names)


['Orfevre', 'Win Variation', 'Gold Ship', 'Love Is Boo Shet', 'Tamamo Best Play', 'Curren Mirotic', 'Desperado', 'To The Glory', 'T M Inazuma', 'Verde Green', 'Admire Rakti', 'Lovely Day', 'Nakayama Knight', 'Tosen Jordan', 'Danon Ballade', 'Lelouch']


In [49]:
df.head()

,Race,Video,Fav,Fin.,J,Wgt.,Odds,Rnrs,Bk,Draw,...,Gng.,FT,Mrg.,Pos. at Bends,F3F-L3F,L3F,Horse Wgt.,Winner (Runner-up),Prize (¥ mil),Horse
0,22 Dec 2013 NAK 10R ARIMA KINEN (THE GRAND PRI...,NaN,1.0,1,K.Ikezoe,57.0,1.6,16.0,3.0,6,...,Gd,2:32.3,-1.3,13-13-12-2,30.3-36.7,36.0,466(0),(Win Variation),203.40,Orfevre
1,6 Oct 2013 5R Prix de l'Arc de Triomphe G1,NaN,1.0,2,C.Soumillon,59.5,0.0,17.0,NaN,6,...,Sft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Orfevre
2,15 Sep 2013 6R Prix Foy G2,NaN,1.0,1,C.Soumillon,58.0,0.0,NaN,NaN,7,...,Sft,2:41.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Orfevre
3,31 Mar 2013 HAN 11R SANKEI OSAKA HAI G2,NaN,1.0,1,K.Ikezoe,58.0,1.2,14.0,4.0,5,...,Gd,1:59.0,-0.1,10-11-9-6,36.7-33.7,33.0,464(+6),(Shonan Mighty),60.98,Orfevre
4,25 Nov 2012 TKY 11R JAPAN CUP G1,NaN,1.0,2,K.Ikezoe,57.0,2.0,17.0,8.0,17,...,Gd,2:23.1,0.0,12-13-10-3,35.8-34.7,32.9,458(0),Gentildonna,101.02,Orfevre


In [50]:
col = df.pop("Horse")
df.insert(0, "Horse", col)

In [51]:
df.head()

,Horse,Race,Video,Fav,Fin.,J,Wgt.,Odds,Rnrs,Bk,...,WX,Gng.,FT,Mrg.,Pos. at Bends,F3F-L3F,L3F,Horse Wgt.,Winner (Runner-up),Prize (¥ mil)
0,Orfevre,22 Dec 2013 NAK 10R ARIMA KINEN (THE GRAND PRI...,NaN,1.0,1,K.Ikezoe,57.0,1.6,16.0,3.0,...,NaN,Gd,2:32.3,-1.3,13-13-12-2,30.3-36.7,36.0,466(0),(Win Variation),203.40
1,Orfevre,6 Oct 2013 5R Prix de l'Arc de Triomphe G1,NaN,1.0,2,C.Soumillon,59.5,0.0,17.0,NaN,...,NaN,Sft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Orfevre,15 Sep 2013 6R Prix Foy G2,NaN,1.0,1,C.Soumillon,58.0,0.0,NaN,NaN,...,NaN,Sft,2:41.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Orfevre,31 Mar 2013 HAN 11R SANKEI OSAKA HAI G2,NaN,1.0,1,K.Ikezoe,58.0,1.2,14.0,4.0,...,NaN,Gd,1:59.0,-0.1,10-11-9-6,36.7-33.7,33.0,464(+6),(Shonan Mighty),60.98
4,Orfevre,25 Nov 2012 TKY 11R JAPAN CUP G1,NaN,1.0,2,K.Ikezoe,57.0,2.0,17.0,8.0,...,NaN,Gd,2:23.1,0.0,12-13-10-3,35.8-34.7,32.9,458(0),Gentildonna,101.02


In [52]:
cols_to_drop = ["Video", "Odds", "Fav", "Winner (Runner-up)", "Prize (¥ mil)", "WX"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


In [53]:
df.rename(columns={
    "J": "Jockey",
    "Fin.": "Finish",
    "Wgt.": "Weight",
    "Rnrs": "Runners",
    "Bk": "Block",
    "Gng.": "Ground",
    "Mrg.": "Margin",
    "Horse Wgt.": "Horse Weight"
}, inplace = True)

In [54]:
df['Date'] = df['Race'].str.extract(r'^(\d{1,2}\s+\w+)')     # e.g. "15 Sep"
df['Year'] = df['Race'].str.extract(r'\b(\d{4})\b') # e.g. "2013"
df['RaceDate'] = pd.to_datetime(df['Date'] + ' ' + df['Year'], format='%d %b %Y', errors='coerce')
df['Race Details'] = df['Race'].str.extract(r'\d{4}\s+(.*)')

In [55]:
df['Year'] = pd.to_numeric(df['Year'], errors="coerce")

"""Data Cleaning"""

In [56]:
df.isna().sum()

Horse             0
Race              0
Finish            0
Jockey            5
Weight            0
Runners           7
Block            13
Draw              0
Dis.              0
Ground            0
FT                8
Margin           14
Pos. at Bends    14
F3F-L3F          13
L3F              14
Horse Weight     13
Date              0
Year              0
RaceDate          0
Race Details      0
dtype: int64

In [57]:
print(len(df))
df = df[df['Year'] <= 2013]

499


In [58]:
df = df[df['RaceDate'] != '22 Dec 2013']

In [59]:
df.isna().sum()

Horse            0
Race             0
Finish           0
Jockey           5
Weight           0
Runners          3
Block            6
Draw             0
Dis.             0
Ground           0
FT               5
Margin           7
Pos. at Bends    7
F3F-L3F          6
L3F              7
Horse Weight     7
Date             0
Year             0
RaceDate         0
Race Details     0
dtype: int64

In [60]:
df = df.dropna(subset=['Margin'])

In [61]:
df.isna().sum()

Horse            0
Race             0
Finish           0
Jockey           3
Weight           0
Runners          0
Block            0
Draw             0
Dis.             0
Ground           0
FT               0
Margin           0
Pos. at Bends    0
F3F-L3F          0
L3F              0
Horse Weight     0
Date             0
Year             0
RaceDate         0
Race Details     0
dtype: int64

In [62]:
df[df['Jockey'].isna()]

,Horse,Race,Finish,Jockey,Weight,Runners,Block,Draw,Dis.,Ground,FT,Margin,Pos. at Bends,F3F-L3F,L3F,Horse Weight,Date,Year,RaceDate,Race Details
245,To The Glory,11 Dec 2010 KOK 11R CHUNICHI SHIMBUN HAI G3,1,NaN,55.0,18.0,1.0,1,T2000,Gd,1:58.7,-0.3,8-10-3-2,34.6-33.9,33.6,528(+2),11 Dec,2010,2010-12-11,KOK 11R CHUNICHI SHIMBUN HAI G3
246,To The Glory,21 Nov 2010 KYT 11R MILE CHAMPIONSHIP G1,7,NaN,56.0,18.0,5.0,9,T1600,Gd,1:32.3,0.5,16-17,33.7-35.1,34.2,526(+6),21 Nov,2010,2010-11-21,KYT 11R MILE CHAMPIONSHIP G1
470,Danon Ballade,10 Dec 2011 KOK 11R CHUNICHI SHIMBUN HAI G3,3,NaN,56.0,18.0,7.0,13,T2000,GS,1:59.6,0.0,7-8-5-2,35.1-36.0,35.6,480(+2),10 Dec,2011,2011-12-10,KOK 11R CHUNICHI SHIMBUN HAI G3


In [63]:
# Fill missing jockeys with the horse's most frequent one
df['Jockey'] = df.groupby('Horse')['Jockey'].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)

df.isna().sum()

Horse            0
Race             0
Finish           0
Jockey           0
Weight           0
Runners          0
Block            0
Draw             0
Dis.             0
Ground           0
FT               0
Margin           0
Pos. at Bends    0
F3F-L3F          0
L3F              0
Horse Weight     0
Date             0
Year             0
RaceDate         0
Race Details     0
dtype: int64

In [64]:
df["Track"] = df["Dis."].str[0].map({"T": "Turf", "D": "Dirt"})
df["Distance"] = df["Dis."].str[1:].astype(int)

In [65]:
df['Finish'] = pd.to_numeric(df['Finish'], errors='coerce')
df = df.dropna(subset=['Finish'])

In [66]:
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        parts = time_str.split(":")
        if len(parts) == 2:
            minutes = int(parts[0])
            seconds = float(parts[1])
            return minutes*60 + seconds
        else:
            return float(parts[0])
    except:
        return None
           

In [67]:
def rolling_past_avg(x):
    # reverse each group, apply rolling, then reverse back
    return (
        x[::-1]  # reverse order so rolling goes from oldest → newest
        .shift(1)
        .rolling(window=3, min_periods=1)
        .mean()
    )[::-1]  # reverse back to original order




In [68]:
def extract_corner_features(corner_str, field_size=None):
    """Extract early position, final corner position, and position change features from corner string."""
    try:
        pos = [int(x) for x in str(corner_str).split('-') if x.isdigit()]
        early = pos[0]
        final = pos[-1]
        change = early - final  # positive = moved up, negative = faded
        return pd.Series({
            'EarlyPos': early,
            'FinalCornerPos': final,
            'PosChange': change,
            'PassingRate': change / field_size if field_size and field_size > 0 else np.nan
        })
    except Exception:
        return pd.Series({
            'EarlyPos': np.nan,
            'FinalCornerPos': np.nan,
            'PosChange': np.nan,
            'PassingRate': np.nan
        })



In [69]:
df["FT_seconds"] = df["FT"].apply(time_to_seconds)
print(df["FT_seconds"].head(5))

3    119.0
4    143.1
7    130.9
8    195.6
9    191.9
Name: FT_seconds, dtype: float64


In [70]:
df['Win'] = (df['Finish'] == 1).astype(int)
df['Top3'] = (df['Finish'] <= 3).astype(int)

In [71]:
df['AvgPlacement_Last3'] = (
    df.groupby('Horse', group_keys=False)['Finish']
      .transform(rolling_past_avg)
)



In [72]:
df['HorseAvgFinish'] = df.groupby('Horse')['Finish'].transform('mean')
df['AvgPlacement_Last3'] = df['AvgPlacement_Last3'].fillna(df['HorseAvgFinish'])

In [73]:
corner_features = df.apply(
    lambda row: extract_corner_features(row['Pos. at Bends'], row['Runners']),
    axis=1
)

df = pd.concat([df, corner_features], axis=1)


In [74]:
# Extract 3-letter course/track code
df['Course'] = df['Race Details'].str.extract(r'^([A-Z]{3})')

# Extract race grade (G1, G2, G3, or Listed)
df['RaceGrade'] = df['Race'].str.extract(r'\b(G\d|Listed)\b', flags=re.IGNORECASE)
df['RaceGrade'] = df['RaceGrade'].str.upper().fillna('UNGRADED')

In [75]:
grade_map = {
    'G1': 3,
    'G2': 2,
    'G3': 1,
    'LISTED': 0.5,
    'UNGRADED': 0
}
df['RaceGrade_numeric'] = df['RaceGrade'].map(grade_map)

print(df[['Race Details', 'Course', 'RaceGrade', 'RaceGrade_numeric']].head())

                    Race Details Course RaceGrade  RaceGrade_numeric
3    HAN 11R SANKEI OSAKA HAI G2    HAN        G2                2.0
4           TKY 11R JAPAN CUP G1    TKY        G1                3.0
7    HAN 11R TAKARAZUKA KINEN G1    HAN        G1                3.0
8  KYT 11R TENNO SHO (SPRING) G1    KYT        G1                3.0
9   HAN 11R HANSHIN DAISHOTEN G2    HAN        G2                2.0


In [76]:
# --- 6️⃣ Compute Days Since Last Race per Horse ---
df = df.sort_values(['Horse', 'RaceDate'])

df['DaysSinceLastRace'] = df.groupby('Horse')['RaceDate'].diff().dt.days
df['DaysSinceLastRace'] = df['DaysSinceLastRace'].fillna(df['DaysSinceLastRace'].median())

In [77]:
df['Speed_mps'] = df['Distance'] / df['FT_seconds']  # meters per second
df['Speed_kph'] = df['Speed_mps'] * 3.6  # convert to km/h

In [78]:
def extract_race_name(race_str):
    """Extract race name after 'R ' and remove the final word (grade)."""
    if not isinstance(race_str, str) or "R " not in race_str:
        return None  # handle missing or malformed data
    
    # take everything after 'R '
    name_part = race_str.split("R ", 1)[1].strip()
    parts = name_part.split()
    
    # remove the last word if it looks like a grade or short tag (e.g. G1, Listed, Open)
    if len(parts) > 1 and len(parts[-1]) <= 7:
        parts = parts[:-1]
    
    # rejoin cleaned name
    clean_name = " ".join(parts)
    
    # optional: title-case for presentation
    return clean_name.title()

# Apply to your DataFrame
df["Race Name"] = df["Race Details"].apply(extract_race_name)

print(df[["Race Details", "Race Name"]])

                              Race Details                  Race Name
349                         KYT 5R ２yo DBT                        ２Yo
348                         HAN 4R ２yo MDN                        ２Yo
347  KYT 9R FUKUJUSO TOKUBETSU ALW (1 Win)  Fukujuso Tokubetsu Alw (1
346               HAN 10R WAKABA STAKES OP              Wakaba Stakes
345                 HAN 6R ３yo ALW (1 Win)                 ３Yo Alw (1
..                                     ...                        ...
33                  KYT 11R KYOTO KINEN G2                Kyoto Kinen
32                   NAK 11R NIKKEI SHO G2                 Nikkei Sho
31           KYT 11R TENNO SHO (SPRING) G1         Tenno Sho (Spring)
30             HAN 11R TAKARAZUKA KINEN G1           Takarazuka Kinen
29                    CHU 11R KINKO SHO G2                  Kinko Sho

[302 rows x 2 columns]


In [79]:
df.head(10)

,Horse,Race,Finish,Jockey,Weight,Runners,Block,Draw,Dis.,Ground,...,FinalCornerPos,PosChange,PassingRate,Course,RaceGrade,RaceGrade_numeric,DaysSinceLastRace,Speed_mps,Speed_kph,Race Name
349,Admire Rakti,14 Nov 2010 KYT 5R ２yo DBT,2,K.Fujioka,55.0,9.0,3.0,3,T2000,Gd,...,4.0,4.0,0.444444,KYT,UNGRADED,0.0,35.0,16.051364,57.784912,２Yo
348,Admire Rakti,11 Dec 2010 HAN 4R ２yo MDN,1,Y.Iwata,55.0,13.0,6.0,9,T1800,Gd,...,10.0,1.0,0.076923,HAN,UNGRADED,0.0,27.0,16.651249,59.944496,２Yo
347,Admire Rakti,8 Jan 2011 KYT 9R FUKUJUSO TOKUBETSU ALW (1 Win),6,Y.Iwata,56.0,14.0,7.0,11,T2000,Gd,...,12.0,0.0,0.000000,KYT,UNGRADED,0.0,28.0,16.460905,59.259259,Fukujuso Tokubetsu Alw (1
346,Admire Rakti,20 Mar 2011 HAN 10R WAKABA STAKES OP,5,Y.Fujioka,56.0,16.0,3.0,5,T2000,Gd,...,12.0,3.0,0.187500,HAN,UNGRADED,0.0,71.0,16.680567,60.050042,Wakaba Stakes
345,Admire Rakti,2 Apr 2011 HAN 6R ３yo ALW (1 Win),10,Y.Fujioka,56.0,18.0,3.0,5,T2400,Gd,...,14.0,3.0,0.166667,HAN,UNGRADED,0.0,13.0,16.326531,58.775510,３Yo Alw (1
344,Admire Rakti,24 Sep 2011 HAN 8R ３yo+ ALW (1 Win),1,Y.Iwata,54.0,14.0,6.0,9,T2000,Gd,...,11.0,0.0,0.000000,HAN,UNGRADED,0.0,175.0,16.488046,59.356966,３Yo+ Alw (1
343,Admire Rakti,22 Oct 2011 KYT 9R KITANO TOKUBETSU ALW (2 Win),2,Y.Fujioka,55.0,7.0,5.0,5,T2000,Gd,...,5.0,1.0,0.142857,KYT,UNGRADED,0.0,28.0,16.891892,60.810811,Kitano Tokubetsu Alw (2
342,Admire Rakti,13 Nov 2011 KYT 10R DONCASTER CUP ALW (2 Win),3,Y.Kawada,55.0,10.0,3.0,3,T2000,Gd,...,9.0,0.0,0.000000,KYT,UNGRADED,0.0,22.0,16.722408,60.200669,Doncaster Cup Alw (2
341,Admire Rakti,10 Dec 2011 HAN 9R SAKAIMINATO TOKUBETSU ALW (...,3,I.Mendizabal,55.0,7.0,3.0,3,T2200,Gd,...,6.0,1.0,0.142857,HAN,UNGRADED,0.0,27.0,16.129032,58.064516,Sakaiminato Tokubetsu Alw (2
340,Admire Rakti,25 Feb 2012 HAN 9R SENRIYAMA TOKUBETSU ALW (2 ...,2,Y.Iwata,56.0,13.0,5.0,6,T2000,Gd,...,8.0,0.0,0.000000,HAN,UNGRADED,0.0,77.0,16.273393,58.584215,Senriyama Tokubetsu Alw (2


In [80]:
df.columns

Index(['Horse', 'Race', 'Finish', 'Jockey', 'Weight', 'Runners', 'Block',
       'Draw', 'Dis.', 'Ground', 'FT', 'Margin', 'Pos. at Bends', 'F3F-L3F',
       'L3F', 'Horse Weight', 'Date', 'Year', 'RaceDate', 'Race Details',
       'Track', 'Distance', 'FT_seconds', 'Win', 'Top3', 'AvgPlacement_Last3',
       'HorseAvgFinish', 'EarlyPos', 'FinalCornerPos', 'PosChange',
       'PassingRate', 'Course', 'RaceGrade', 'RaceGrade_numeric',
       'DaysSinceLastRace', 'Speed_mps', 'Speed_kph', 'Race Name'],
      dtype='object')

In [84]:
print(df['Race Name'].head())

349                          ２Yo
348                          ２Yo
347    Fukujuso Tokubetsu Alw (1
346                Wakaba Stakes
345                   ３Yo Alw (1
Name: Race Name, dtype: object


In [81]:
df.to_excel(r"C:\Users\User\Documents\horse-racing-arima-kinen-2013\data\processed\cleaned.xlsx")